Transformation Layer


In [2]:
import pandas as pd
import pymysql 
from sqlalchemy import create_engine

In [4]:
df = pd.read_csv("universities-clean.csv")
df.head()

,Rank,Institution,City,Country,Continent,Founded,Affiliation,AttendanceMode,Enrollment,Link
0,1,Indira Gandhi National Open University,New Delhi,India,Asia,1985,Public,Distance/In-Person,7140000,https://en.wikipedia.org/wiki/Indira_Gandhi_Na...
1,2,"National University, Bangladesh",Gazipur,Bangladesh,Asia,1992,Public,In-Person,3425832,https://en.wikipedia.org/wiki/National_Univers...
2,3,Anadolu University,Eskişehir,Turkey,Asia,1958,Public,Distance/In-Person,1974343,https://en.wikipedia.org/wiki/Anadolu_University
3,4,California Community Colleges,California,United States,North America,1967,Public,In-Person,1800000,https://en.wikipedia.org/wiki/California_Commu...
4,5,Islamic Azad University,Iran,Iran,Asia,1982,Private,In-Person,1778000,https://en.wikipedia.org/wiki/Islamic_Azad_Uni...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Rank            90 non-null     int64 
 1   Institution     90 non-null     object
 2   City            90 non-null     object
 3   Country         90 non-null     object
 4   Continent       90 non-null     object
 5   Founded         90 non-null     int64 
 6   Affiliation     90 non-null     object
 7   AttendanceMode  90 non-null     object
 8   Enrollment      90 non-null     int64 
 9   Link            90 non-null     object
dtypes: int64(3), object(7)
memory usage: 7.2+ KB


In [6]:
# Create dim_institution
dim_institution = df[['Institution', 'Affiliation', 'AttendanceMode', 'Founded', 'Link']].drop_duplicates().reset_index(drop=True)
dim_institution['institution_id'] = dim_institution.index + 1

In [7]:
# Create dim_location
dim_location = df[['City', 'Country', 'Continent']].drop_duplicates().reset_index(drop=True)
dim_location['location_id'] = dim_location.index + 1

In [8]:
# Merge keys into original DataFrame
df = df.merge(dim_institution, on=['Institution', 'Affiliation', 'AttendanceMode', 'Founded', 'Link'], how='left')
df = df.merge(dim_location, on=['City', 'Country', 'Continent'], how='left')

In [9]:
# Create fact_university_ranking
fact_university_ranking = df[['Rank', 'Enrollment', 'institution_id', 'location_id']].copy()
fact_university_ranking['id'] = fact_university_ranking.index + 1  # Optional primary key


In [10]:
# Reorder columns
fact_university_ranking = fact_university_ranking[['id', 'Rank', 'institution_id', 'location_id', 'Enrollment']]

In [11]:
print("📘 dim_institution:")
print(dim_institution.head())

📘 dim_institution:
                              Institution Affiliation      AttendanceMode  \
0  Indira Gandhi National Open University      Public  Distance/In-Person   
1         National University, Bangladesh      Public           In-Person   
2                      Anadolu University      Public  Distance/In-Person   
3           California Community Colleges      Public           In-Person   
4                 Islamic Azad University     Private           In-Person   

   Founded                                               Link  institution_id  
0     1985  https://en.wikipedia.org/wiki/Indira_Gandhi_Na...               1  
1     1992  https://en.wikipedia.org/wiki/National_Univers...               2  
2     1958   https://en.wikipedia.org/wiki/Anadolu_University               3  
3     1967  https://en.wikipedia.org/wiki/California_Commu...               4  
4     1982  https://en.wikipedia.org/wiki/Islamic_Azad_Uni...               5  


In [12]:
print("\n🌍 dim_location:")
print(dim_location.head())


🌍 dim_location:
         City        Country      Continent  location_id
0   New Delhi          India           Asia            1
1     Gazipur     Bangladesh           Asia            2
2   Eskişehir         Turkey           Asia            3
3  California  United States  North America            4
4        Iran           Iran           Asia            5


In [13]:
print("\n📊 fact_university_ranking:")
print(fact_university_ranking.head())


📊 fact_university_ranking:
   id  Rank  institution_id  location_id  Enrollment
0   1     1               1            1     7140000
1   2     2               2            2     3425832
2   3     3               3            3     1974343
3   4     4               4            4     1800000
4   5     5               5            5     1778000


In [14]:
dim_institution.to_csv("dim_institution.csv", index=False)
dim_location.to_csv("dim_location.csv", index=False)
fact_university_ranking.to_csv("fact_university_ranking.csv", index=False)